In [ ]:
from datasets import load_dataset, Dataset
import polars as pl
from IPython.display import display, Markdown, Latex, clear_output
from collections import defaultdict
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from rich.syntax import Syntax
from rich.console import Console
import pandas as pd
from enum import Enum
from functools import partial
%load_ext autoreload
%autoreload 2

In [6]:
import sys
sys.path.append('..')
from evaluation.code_execution import evaluate_sample_codecontests, evaluate_python_code_exec

In [ ]:
# https://huggingface.co/datasets/deepmind/code_contests
cc_dataset = load_dataset("deepmind/code_contests")["train"]

In [ ]:
class Language(Enum):
    UNKNOWN = 0
    PYTHON = 1
    CPP = 2
    PYTHON3 = 3
    JAVA = 4

class ProblemSource(Enum):
    UNKNOWN = 0
    CODECHEF = 1
    CODEFORCES = 2
    HACKEREARTH = 3
    CODEJAM = 4
    ATCODER = 5
    AIZU = 6

class Difficulty(Enum):
    UNKNOWN_DIFFICULTY = 0
    EASY = 1
    MEDIUM = 2
    HARD = 3
    HARDER = 4
    HARDEST = 5
    EXTERNAL = 6
    A = 7
    B = 8
    C = 9
    D = 10
    E = 11
    F = 12
    G = 13
    H = 14
    I = 15
    J = 16
    K = 17
    L = 18
    M = 19
    N = 20
    O = 21
    P = 22
    Q = 23
    R = 24
    S = 25
    T = 26
    U = 27
    V = 28

In [ ]:
console = Console()


def print_code_snippet(snippet: str, console: Console):
    formatted_snippet = Syntax(
        snippet,
        "python",
        theme="monokai",
        line_numbers=True,
    )
    console.print(formatted_snippet)


first_n_samples = []

iterator = tqdm(cc_dataset)
for i, sample in enumerate(iterator):
    description = sample["description"]
    test_inputs = sample["public_tests"]["input"] + sample["private_tests"]["input"]
    test_outputs = sample["public_tests"]["output"] + sample["private_tests"]["output"]
    problem_source = ProblemSource(sample["source"])
    difficulty = Difficulty(sample["difficulty"])

    should_keep_problem = True

    if problem_source != ProblemSource.CODEFORCES:
        should_keep_problem = False

    if sample["is_description_translated"]:  # type: ignore
        should_keep_problem = False

    if sample["input_file"] or sample["output_file"]:  # type: ignore
        should_keep_problem = False

    if len(sample["private_tests"]["input"]) == 0:
        should_keep_problem = False

    # console.clear()
    # console.print(description)
    # display(Markdown(description))

    has_python_solution = False

    for j, (lang_txt, solution_code) in enumerate(
        zip(sample["solutions"]["language"], sample["solutions"]["solution"])
    ):
        language = Language(lang_txt)
        if language == Language.PYTHON3:
            has_python_solution = True
            break

        # solution_code = solution_code.replace("sys.exit", "exit")
        # # clear_output(wait=True)
        # display(f"problem {i}, solution {j}")
        # display(Markdown(f"```python\n{solution_code}\n```"))
        # print_code_snippet(solution_code, console)

        # for inputs, outputs in zip(test_inputs, test_outputs):
        #     # each time input is called, it will return the next input value
        #     out = evaluate_python_code_exec(solution_code, inputs)

    if not has_python_solution:
        should_keep_problem = False

    if should_keep_problem:
        sample_to_keep = {
            "source": problem_source,
            "difficulty": difficulty,
        }
        for key in ("name", "description", "public_tests", "private_tests", "cf_rating", "cf_points"):
            sample_to_keep[key] = sample[key]
        first_n_samples.append(sample_to_keep)
    
    iterator.set_postfix({"n_samples": len(first_n_samples)})

    if len(first_n_samples) > 2500:
        break

first_n_df = pd.DataFrame(first_n_samples)

In [ ]:
for i, problem in enumerate(cc_dataset):
    val = f"### Problem {i}: {problem['name']}\n{problem['description']}"
    display(Markdown(val))

In [ ]:

for col in first_n_df.columns:
    if pd.api.types.is_object_dtype(first_n_df[col]) and isinstance(first_n_df[col].iloc[0], Enum):
        first_n_df[col] = first_n_df[col].apply(lambda x: x.value if isinstance(x, Enum) else x)

Dataset.from_pandas(first_n_df).to_parquet("dataset_samples/codeforces_problems_subset.parquet")

In [ ]:
test_df = Dataset.from_parquet("../dataset_samples/codeforces_problems_subset.parquet")
test_df[0]

In [22]:
test_code = """
def solution(problem_input):
    n, m, k = map(int, problem_input[0].split())
    edges = [list(map(int, line.split())) for line in problem_input[1:]]

    adj = [[] for _ in range(n + 1)]

    results = []
    for i in range(m):
        adj[edges[i][0]].append(edges[i][1])
        adj[edges[i][1]].append(edges[i][0])

        max_people = 0
        for j in range(1 << n):
            group = []
            for bit in range(n):
                if (j >> bit) & 1:
                    group.append(bit + 1)

            valid_group = True
            for person in group:
                friend_count = 0
                for friend in adj[person]:
                    if friend in group:
                        friend_count += 1
                if friend_count < k:
                    valid_group = False
                    break

            if valid_group:
                max_people = max(max_people, len(group))
        results.append(max_people)
    return results
"""

tests = {
    "input": [
        "4 4 2\n2 3\n1 2\n1 3\n1 4\n",
        "5 8 2\n2 1\n4 2\n5 4\n5 2\n4 3\n5 1\n4 1\n3 2\n",
        "5 7 2\n1 5\n3 2\n2 5\n3 4\n1 2\n5 3\n1 3\n",
    ],
    "output": ["0\n0\n3\n3\n", "0\n0\n0\n3\n3\n4\n4\n5\n", "0\n0\n0\n0\n3\n4\n4\n"],
}

for test_input in tests["input"]:
    ti = test_input.strip().split("\n")
    out = evaluate_python_code_exec(test_code + f"solution({ti})")
    print(out)

['passed']
['passed']
['passed']
